In [24]:
# imports
import pandas as pd
import numpy as np
import sys
import os
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler, normalize
import pickle
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

In [2]:
# add paths and import scripts
sys.path.append('.')
sys.path.append('..')
sys.path.insert(1,'../scripts')

# User satisfaction analysis

### Engagement analysis + Experience analysis

* Assign engagement score to each user

In [3]:
user_eng_df = pd.read_csv('../data/user_engagement.csv.bz2')
user_eng_df

,MSISDN/Number,cluster,XDR Sessions,Dur. (ms),total_data,best_cluster
0,3.360100e+10,1,1,116720.0,8.786906e+08,2
1,3.360100e+10,2,1,181230.0,1.568596e+08,3
2,3.360100e+10,2,1,134969.0,5.959665e+08,3
3,3.360101e+10,1,1,49878.0,4.223207e+08,1
4,3.360101e+10,1,2,37104.0,1.457411e+09,2
...,...,...,...,...,...,...
106851,3.379000e+10,1,1,8810.0,7.146416e+08,2
106852,3.379000e+10,2,1,140988.0,4.803073e+08,3
106853,3.197020e+12,1,1,86399.0,2.321240e+08,1
106854,3.370000e+14,1,1,86399.0,5.962878e+08,2


In [4]:
eng_df = user_eng_df.set_index('MSISDN/Number')[['XDR Sessions', 'Dur. (ms)', 'total_data']]
eng_df

,XDR Sessions,Dur. (ms),total_data
MSISDN/Number,,,
3.360100e+10,1,116720.0,8.786906e+08
3.360100e+10,1,181230.0,1.568596e+08
3.360100e+10,1,134969.0,5.959665e+08
3.360101e+10,1,49878.0,4.223207e+08
3.360101e+10,2,37104.0,1.457411e+09
...,...,...,...
3.379000e+10,1,8810.0,7.146416e+08
3.379000e+10,1,140988.0,4.803073e+08
3.197020e+12,1,86399.0,2.321240e+08


In [5]:
# scale data
scaler = StandardScaler()
scaled_data = scaler.fit_transform(eng_df)
scaled_data

array([[-0.12034413, -0.05434193,  0.10890723],
       [-0.12034413,  0.23939902, -0.31753898],
       [-0.12034413,  0.02875338, -0.05812166],
       ...,
       [-0.12034413, -0.19240609, -0.27307401],
       [-0.12034413, -0.19240609, -0.05793181],
       [-0.12034413, -0.19240609, -0.33940905]])

In [6]:
normalized_data = normalize(scaled_data)
normalized_data

array([[-0.70309945, -0.31748768,  0.63628039],
       [-0.28964919,  0.57619537, -0.76426582],
       [-0.88033551,  0.21033535, -0.42516871],
       ...,
       [-0.33893405, -0.54188746, -0.76907847],
       [-0.51380856, -0.82147667, -0.24733954],
       [-0.29475108, -0.47124776, -0.83129258]])

In [7]:
with open("../models/user_engagement_cluster.pkl", "rb") as f:
    k_means1 = pickle.load(f)

In [8]:
less_engaged_cluster = 3

distance = k_means1.fit_transform(normalized_data)
distance_from_less_engaged_cluster = list(
    map(lambda x: x[less_engaged_cluster], distance))
user_eng_df['engagement_score'] = distance_from_less_engaged_cluster
user_eng_df

,MSISDN/Number,cluster,XDR Sessions,Dur. (ms),total_data,best_cluster,engagement_score
0,3.360100e+10,1,1,116720.0,8.786906e+08,2,1.324146
1,3.360100e+10,2,1,181230.0,1.568596e+08,3,0.477645
2,3.360100e+10,2,1,134969.0,5.959665e+08,3,0.532881
3,3.360101e+10,1,1,49878.0,4.223207e+08,1,1.448688
4,3.360101e+10,1,2,37104.0,1.457411e+09,2,1.771222
...,...,...,...,...,...,...,...
106851,3.379000e+10,1,1,8810.0,7.146416e+08,2,1.601323
106852,3.379000e+10,2,1,140988.0,4.803073e+08,3,0.470075
106853,3.197020e+12,1,1,86399.0,2.321240e+08,1,1.196540
106854,3.370000e+14,1,1,86399.0,5.962878e+08,2,1.385484


* Engagement score calculated and assigned

* Assign experience score to each user

In [9]:
user_exp_df = pd.read_csv('../data/user_experience_.csv.bz2')
user_exp_df

,cluster,MSISDN/Number,TCP DL Retrans. Vol (Bytes),TCP UL Retrans. Vol (Bytes),Avg RTT DL (ms),Avg RTT UL (ms),Avg Bearer TP DL (kbps),Avg Bearer TP UL (kbps)
0,2,3.366496e+10,568730.0,20949.5,42.0,5.0,23.0,44.0
1,2,3.368185e+10,568730.0,20949.5,65.0,5.0,16.0,26.0
2,2,3.376063e+10,568730.0,20949.5,45.0,5.0,6.0,9.0
3,2,3.375034e+10,568730.0,20949.5,45.0,5.0,44.0,44.0
4,2,3.369980e+10,568730.0,20949.5,45.0,5.0,6.0,9.0
...,...,...,...,...,...,...,...,...
149995,0,3.366865e+10,16552848.0,162614.0,313.0,9.0,63420.0,1393.0
149996,2,3.365069e+10,568730.0,20949.5,32.0,0.0,52.0,65.0
149997,2,3.366345e+10,568730.0,20949.5,27.0,2.0,23.0,54.0
149998,2,3.362189e+10,568730.0,20949.5,43.0,6.0,43.0,47.0


In [10]:
# get the combined total
user_exp_df['Total Avg RTT (ms)'] = user_exp_df['Avg RTT DL (ms)'] + user_exp_df['Avg RTT UL (ms)']
user_exp_df['Total Avg Bearer TP (kbps)'] = user_exp_df['Avg Bearer TP DL (kbps)'] + user_exp_df['Avg Bearer TP UL (kbps)']
user_exp_df['Total TCP Retrans. Vol (Bytes)'] = user_exp_df['TCP DL Retrans. Vol (Bytes)'] + user_exp_df['TCP UL Retrans. Vol (Bytes)']

In [11]:
user_exp_df

,cluster,MSISDN/Number,TCP DL Retrans. Vol (Bytes),TCP UL Retrans. Vol (Bytes),Avg RTT DL (ms),Avg RTT UL (ms),Avg Bearer TP DL (kbps),Avg Bearer TP UL (kbps),Total Avg RTT (ms),Total Avg Bearer TP (kbps),Total TCP Retrans. Vol (Bytes)
0,2,3.366496e+10,568730.0,20949.5,42.0,5.0,23.0,44.0,47.0,67.0,589679.5
1,2,3.368185e+10,568730.0,20949.5,65.0,5.0,16.0,26.0,70.0,42.0,589679.5
2,2,3.376063e+10,568730.0,20949.5,45.0,5.0,6.0,9.0,50.0,15.0,589679.5
3,2,3.375034e+10,568730.0,20949.5,45.0,5.0,44.0,44.0,50.0,88.0,589679.5
4,2,3.369980e+10,568730.0,20949.5,45.0,5.0,6.0,9.0,50.0,15.0,589679.5
...,...,...,...,...,...,...,...,...,...,...,...
149995,0,3.366865e+10,16552848.0,162614.0,313.0,9.0,63420.0,1393.0,322.0,64813.0,16715462.0
149996,2,3.365069e+10,568730.0,20949.5,32.0,0.0,52.0,65.0,32.0,117.0,589679.5
149997,2,3.366345e+10,568730.0,20949.5,27.0,2.0,23.0,54.0,29.0,77.0,589679.5
149998,2,3.362189e+10,568730.0,20949.5,43.0,6.0,43.0,47.0,49.0,90.0,589679.5


In [12]:
exp_df = user_exp_df.set_index('MSISDN/Number')[
    ['Total Avg RTT (ms)', 'Total Avg Bearer TP (kbps)', 'Total TCP Retrans. Vol (Bytes)']]

In [13]:
# scale data
scaler = StandardScaler()
scaled_data = scaler.fit_transform(exp_df)
scaled_data

array([[-0.11677969, -0.55929365, -0.07241413],
       [-0.0761391 , -0.56022559, -0.07241413],
       [-0.11147874, -0.56123209, -0.07241413],
       ...,
       [-0.14858536, -0.55892088, -0.07241413],
       [-0.11324572, -0.55843627, -0.07241413],
       [-0.1256146 , -0.55914454, -0.07241413]])

In [14]:
normalized_data = normalize(scaled_data)
normalized_data

array([[-0.20276858, -0.97112078, -0.12573515],
       [-0.13357866, -0.98286147, -0.12704357],
       [-0.19328425, -0.97307636, -0.12555319],
       ...,
       [-0.25492905, -0.95894485, -0.12424149],
       [-0.19715956, -0.97223141, -0.12607221],
       [-0.21746255, -0.96798465, -0.12536252]])

In [15]:
with open("../models/user_exp_cluster.pkl", "rb") as f:
    k_means2 = pickle.load(f)

In [16]:
less_experience_cluster = 1

distance = k_means2.fit_transform(normalized_data)
distance_from_less_experience_cluster = list(
    map(lambda x: x[less_experience_cluster], distance))
user_exp_df['experience_score'] = distance_from_less_experience_cluster
user_exp_df

,cluster,MSISDN/Number,TCP DL Retrans. Vol (Bytes),TCP UL Retrans. Vol (Bytes),Avg RTT DL (ms),Avg RTT UL (ms),Avg Bearer TP DL (kbps),Avg Bearer TP UL (kbps),Total Avg RTT (ms),Total Avg Bearer TP (kbps),Total TCP Retrans. Vol (Bytes),experience_score
0,2,3.366496e+10,568730.0,20949.5,42.0,5.0,23.0,44.0,47.0,67.0,589679.5,0.024297
1,2,3.368185e+10,568730.0,20949.5,65.0,5.0,16.0,26.0,70.0,42.0,589679.5,0.080388
2,2,3.376063e+10,568730.0,20949.5,45.0,5.0,6.0,9.0,50.0,15.0,589679.5,0.028937
3,2,3.375034e+10,568730.0,20949.5,45.0,5.0,44.0,44.0,50.0,88.0,589679.5,0.028064
4,2,3.369980e+10,568730.0,20949.5,45.0,5.0,6.0,9.0,50.0,15.0,589679.5,0.028937
...,...,...,...,...,...,...,...,...,...,...,...,...
149995,0,3.366865e+10,16552848.0,162614.0,313.0,9.0,63420.0,1393.0,322.0,64813.0,16715462.0,1.980757
149996,2,3.365069e+10,568730.0,20949.5,32.0,0.0,52.0,65.0,32.0,117.0,589679.5,0.044104
149997,2,3.366345e+10,568730.0,20949.5,27.0,2.0,23.0,54.0,29.0,77.0,589679.5,0.051321
149998,2,3.362189e+10,568730.0,20949.5,43.0,6.0,43.0,47.0,49.0,90.0,589679.5,0.026448


* Experience score calculated and assigned

###  Top 10 satisfied customers based on the average of engagement & experience scores

In [17]:
# rename column names for the satisfaction score dataframe merger
user_eng_df.rename(columns={'cluster': 'engagement_cluster'}, inplace=True)
user_exp_df.rename(columns={'cluster': 'experience_cluster'}, inplace=True)

In [18]:
# get the user satisfaction data frame
user_satisfaction_df = pd.merge(user_eng_df, user_exp_df, on='MSISDN/Number')
user_satisfaction_df

,MSISDN/Number,engagement_cluster,XDR Sessions,Dur. (ms),total_data,best_cluster,engagement_score,experience_cluster,TCP DL Retrans. Vol (Bytes),TCP UL Retrans. Vol (Bytes),Avg RTT DL (ms),Avg RTT UL (ms),Avg Bearer TP DL (kbps),Avg Bearer TP UL (kbps),Total Avg RTT (ms),Total Avg Bearer TP (kbps),Total TCP Retrans. Vol (Bytes),experience_score
0,3.360100e+10,1,1,116720.0,8.786906e+08,2,1.324146,2,568730.0,20949.5,46.0,0.0,37.0,39.0,46.0,76.0,589679.5,0.023498
1,3.360100e+10,2,1,181230.0,1.568596e+08,3,0.477645,2,568730.0,20949.5,30.0,1.0,48.0,51.0,31.0,99.0,589679.5,0.046433
2,3.360100e+10,2,1,134969.0,5.959665e+08,3,0.532881,2,568730.0,20949.5,45.0,5.0,48.0,49.0,50.0,97.0,589679.5,0.027957
3,3.360101e+10,1,1,49878.0,4.223207e+08,1,1.448688,2,1066.0,20949.5,69.0,15.0,204.0,44.0,84.0,248.0,22015.5,0.119920
4,3.360101e+10,1,2,37104.0,1.457411e+09,2,1.771222,1,9349630.0,21202.0,86.0,5.0,40372.0,16409.0,91.0,56781.0,9370832.0,1.964792
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
149995,3.379000e+10,1,1,8810.0,7.146416e+08,2,1.601323,2,215044.0,3001.0,42.0,10.0,9978.0,387.0,52.0,10365.0,218045.0,0.382731
149996,3.379000e+10,2,1,140988.0,4.803073e+08,3,0.470075,2,568730.0,20949.5,34.0,6.0,68.0,48.0,40.0,116.0,589679.5,0.026897
149997,3.197020e+12,1,1,86399.0,2.321240e+08,1,1.196540,2,568730.0,20949.5,45.0,5.0,1.0,0.0,50.0,1.0,589679.5,0.029105
149998,3.370000e+14,1,1,86399.0,5.962878e+08,2,1.385484,1,568730.0,20949.5,45.0,5.0,11.0,22.0,50.0,33.0,589679.5,0.028721


In [19]:
# get the satisfaction score based on the average of the engagement and experience scores
user_satisfaction_df['satisfaction_score'] = (
    user_satisfaction_df['engagement_score'] + user_satisfaction_df['experience_score'])/2
user_satisfaction_df

,MSISDN/Number,engagement_cluster,XDR Sessions,Dur. (ms),total_data,best_cluster,engagement_score,experience_cluster,TCP DL Retrans. Vol (Bytes),TCP UL Retrans. Vol (Bytes),Avg RTT DL (ms),Avg RTT UL (ms),Avg Bearer TP DL (kbps),Avg Bearer TP UL (kbps),Total Avg RTT (ms),Total Avg Bearer TP (kbps),Total TCP Retrans. Vol (Bytes),experience_score,satisfaction_score
0,3.360100e+10,1,1,116720.0,8.786906e+08,2,1.324146,2,568730.0,20949.5,46.0,0.0,37.0,39.0,46.0,76.0,589679.5,0.023498,0.673822
1,3.360100e+10,2,1,181230.0,1.568596e+08,3,0.477645,2,568730.0,20949.5,30.0,1.0,48.0,51.0,31.0,99.0,589679.5,0.046433,0.262039
2,3.360100e+10,2,1,134969.0,5.959665e+08,3,0.532881,2,568730.0,20949.5,45.0,5.0,48.0,49.0,50.0,97.0,589679.5,0.027957,0.280419
3,3.360101e+10,1,1,49878.0,4.223207e+08,1,1.448688,2,1066.0,20949.5,69.0,15.0,204.0,44.0,84.0,248.0,22015.5,0.119920,0.784304
4,3.360101e+10,1,2,37104.0,1.457411e+09,2,1.771222,1,9349630.0,21202.0,86.0,5.0,40372.0,16409.0,91.0,56781.0,9370832.0,1.964792,1.868007
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
149995,3.379000e+10,1,1,8810.0,7.146416e+08,2,1.601323,2,215044.0,3001.0,42.0,10.0,9978.0,387.0,52.0,10365.0,218045.0,0.382731,0.992027
149996,3.379000e+10,2,1,140988.0,4.803073e+08,3,0.470075,2,568730.0,20949.5,34.0,6.0,68.0,48.0,40.0,116.0,589679.5,0.026897,0.248486
149997,3.197020e+12,1,1,86399.0,2.321240e+08,1,1.196540,2,568730.0,20949.5,45.0,5.0,1.0,0.0,50.0,1.0,589679.5,0.029105,0.612822
149998,3.370000e+14,1,1,86399.0,5.962878e+08,2,1.385484,1,568730.0,20949.5,45.0,5.0,11.0,22.0,50.0,33.0,589679.5,0.028721,0.707103


In [22]:
user_satisfaction_df_ = user_satisfaction_df[['MSISDN/Number', 'engagement_score', 'experience_score', 'satisfaction_score']]
user_satisfaction_df_.set_index('MSISDN/Number', inplace=True)
user_satisfaction_df_

,engagement_score,experience_score,satisfaction_score
MSISDN/Number,,,
3.360100e+10,1.324146,0.023498,0.673822
3.360100e+10,0.477645,0.046433,0.262039
3.360100e+10,0.532881,0.027957,0.280419
3.360101e+10,1.448688,0.119920,0.784304
3.360101e+10,1.771222,1.964792,1.868007
...,...,...,...
3.379000e+10,1.601323,0.382731,0.992027
3.379000e+10,0.470075,0.026897,0.248486
3.197020e+12,1.196540,0.029105,0.612822


* these are the top 10 satisfied users

In [23]:
top10_satisfied = user_satisfaction_df_.sort_values('satisfaction_score', ascending=False).head(10)
top10_satisfied

,engagement_score,experience_score,satisfaction_score
MSISDN/Number,,,
3.364089e+10,1.808128,1.982931,1.895530
3.369990e+10,1.813490,1.975987,1.894738
3.363798e+10,1.815188,1.973764,1.894476
3.368254e+10,1.811325,1.975796,1.893561
3.376246e+10,1.805588,1.981288,1.893438
3.364120e+10,1.804714,1.981938,1.893326
3.367029e+10,1.806861,1.978815,1.892838
3.366556e+10,1.812693,1.972462,1.892577
3.366639e+10,1.805875,1.978467,1.892171


### Regression model

In [25]:
# splitting training and testing data
X = user_satisfaction_df_[['engagement_score', 'experience_score']]
y = user_satisfaction_df_[['satisfaction_score']]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [27]:
# fit a simple linear regression model
model = LinearRegression()
model.fit(X_train, y_train)

LinearRegression()

In [30]:
y_pred = model.predict(X_test)
print(f"Mean squared error: {np.mean((y_pred - y_test.values) ** 2)}")

Mean squared error: 6.3156978295831845e-31


### k-means

### Aggregate the average satisfaction & experience score per cluster

### Exporting final table containing all user id + engagement, experience & satisfaction scores to local sql db

### Model deployment tracking